Import package

In [3]:
from ensbinclass import preprocessing as pre
from ensbinclass import featureSelection as fs
from ensbinclass import classifier as cl
from ensbinclass import ensemble as ens

In [4]:
import numpy as np
np.random.seed(42)

In [7]:
pr = pre.DataPreprocessing()
pr.load_data('test_data/exampleData_TCGA_LUAD_2000.csv')
pr.show_data()

All columns are numeric
   class      SFTPC     SLC6A4     CLDN18       AGER     ITLN2       LGI3  \
0      0  19.565434  11.948064  14.314268  15.756623  9.202700  11.049183   
1      0  18.824004  12.127599  14.107543  14.666530  7.496016  10.956929   
2      0  18.541982  11.717890  13.501939  14.938679  8.440654   9.740685   
3      0  18.834542   8.590520  14.136818  14.011273  7.259753  10.837580   
4      0  18.988565  10.902216  13.828533  15.863138  9.767983  11.351842   

   C13orf36    FAM83A      FABP4  ...      NEXN     RASSF8      NKD1  \
0  7.818923  4.642644  12.009267  ...  9.059099   9.661631  7.501577   
1  5.994559  4.397002  13.407288  ...  8.334598   9.468622  5.375092   
2  7.331127  1.885184  13.957463  ...  9.298702   9.831359  7.455912   
3  4.755828  3.122491   8.505092  ...  7.241495   9.734491  4.948227   
4  9.122156  5.235413  10.818281  ...  9.521703  10.004926  8.256334   

     PLEKHH2      EDN2     WDR86    CHRNB4     RNF128  C11orf21     APOLD1  
0  

In [6]:
pr = pre.DataPreprocessing()
pr.load_data('test_data/data.train.SM_by_sex.csv')
pr.show_data()

All columns are numeric
   Unnamed: 0  class   X2315252   X2315374    X2315586    X2315638  \
0         813      0  25.540810  69.119637  147.258072  198.511459   
1         812      0  18.860638  54.090511  163.153717  204.175598   
2         806      0  15.028817  47.893856   92.686630  228.351349   
3         800      0  17.168375  58.677734  143.324387  168.731659   
4         798      0  14.756066  76.470932  175.490799  204.933823   

     X2315690    X2315741    X2315896   X2315938  ...   X3931641    X3948938  \
0  124.505341  155.552521  225.493805  55.405968  ...  37.495464  167.333923   
1  134.311157  157.616333  267.594757  75.593033  ...  35.547607  157.591126   
2  106.082451  119.468994  192.104782  44.246822  ...  34.991562  119.812981   
3  114.860771  126.148117  207.840530  62.924339  ...  29.837648  128.944046   
4  139.986298  154.221649  216.028702  76.746902  ...  38.587887  124.418228   

    X3981751   X4000486   X4017141   X4036313    X4038025    X4038031  \
0

In [ ]:
X, y = pr.set_target('class')

In [ ]:
pr.remove_collinear_features(threshold=0.75)

In [ ]:
X.head()

In [ ]:
X = pr.normalization()
X.head()

In [ ]:
y.head()

Feature selection

LASSO

In [ ]:
lasso_features = fs.FeatureSelection(
    X, 
    y,
    method_='lasso',
    size=100,
    params={
        'alpha': 0.1,
        'fit_intercept': True,
        'precompute': False,
        'max_iter': 10000,
        'tol': 0.0001,
        'selection': 'cyclic',
        'random_state': 42,
    },
)

ReliefF

In [ ]:
relieff_features = fs.FeatureSelection(
    X,
    y,
    method_='relieff',
    size=100,
    params={
        'n_neighbors': 100,
        'n_features_to_keep': 100,
    },
)

Get profile

In [ ]:
fs.get_profile(
    return_dataframe=True,
    organism='hsapiens',
    query=lasso_features.features[:5],
)

Classifier

In [ ]:
clf = cl.Classifier(
    X,
    y,
    features=[
        relieff_features.features,
        lasso_features.features,
    ],
    classifiers=[
        'svm',
        'adaboost',
        'random_forest',
    ],
    classifier_params=[
        {'svm': {
            'C': 1, 'kernel': 'linear', 'gamma': 'auto'
            }
        },
        {'adaboost': {
            'n_estimators': 100, 'learning_rate': 0.9
            }
        },
        {'random_forest': {
            'n_estimators': 100, 'criterion': 'gini', 'max_depth': None
            }
        },
    ],
    cv='stratified_k_fold',
    cv_params={'n_splits': 10},
    repetitions=10,
)    

In [ ]:
clf.all_metrics()

In [ ]:
clf.plot_all()

Ensemble

In [ ]:
ens_voting = ens.Ensemble(
    X,
    y,
    features=[
        relieff_features.features,
        lasso_features.features,
    ],
    classifiers=[
        'adaboost',
        'random_forest',
        'svm',
    ],
    classifier_params=[
        {'adaboost': {
            'n_estimators': 100, 'learning_rate': 0.9,
            }
        },
        {'random_forest': {
            'n_estimators': 100, 'criterion': 'gini', 'max_depth': None,
            }
        },
        {'svm': {
            'C': 1, 'kernel': 'linear', 'gamma': 'auto'
            }
        },
    ],  
    cv='stratified_k_fold',
    cv_params={'n_splits': 10},
    ensemble=[
        'voting',
    ],
    ensemble_params=[
        {'voting': {
            'voting': 'soft'
            }
        },
    ],
    repetitions=10,
)

In [ ]:
ens_voting.all_metrics()

In [ ]:
ens_voting.plot_all()

In [ ]:
ens_bagging = ens.Ensemble(
    X,
    y,
    features=[
        relieff_features.features,
        lasso_features.features,
    ],
    classifiers=[
        'adaboost',
        'random_forest',
        'svm',
    ],
    classifier_params=[
        {'adaboost': {
            'n_estimators': 100, 'learning_rate': 0.9,
            }
        },
        {'random_forest': {
            'n_estimators': 100, 'criterion': 'gini', 'max_depth': None,
            }
        },
        {'svm': {
            'C': 1, 'kernel': 'linear', 'gamma': 'auto'
            }
        },
    ],  
    cv='stratified_k_fold',
    cv_params={'n_splits': 10},
    ensemble=[
        'bagging',
    ],
    ensemble_params=[
        {'bagging': {
            'estimator_name': 'random_forest', 'n_estimators': 100, 'max_samples': 0.5, 'max_features': 0.5
            }
        },
    ],
    repetitions=10,
)

In [ ]:
ens_bagging.all_metrics()

In [ ]:
ens_bagging.plot_all()

In [ ]:
ens_stacking = ens.Ensemble(
    X,
    y,
    features=[
        relieff_features.features,
        lasso_features.features,
    ],
    classifiers=[
        'adaboost',
        'random_forest',
        'svm',
    ],
    classifier_params=[
        {'adaboost': {
            'n_estimators': 100, 'learning_rate': 0.9,
            }
        },
        {'random_forest': {
            'n_estimators': 100, 'criterion': 'gini', 'max_depth': None,
            }
        },
        {'svm': {
            'C': 1, 'kernel': 'linear', 'gamma': 'auto'
            }
        },
    ],  
    cv='stratified_k_fold',
    cv_params={'n_splits': 10},
    ensemble=[
        'stacking',
    ],
    ensemble_params=[
        {'stacking': {
            'final_estimator': None,
            }
        },
    ],
    repetitions=10,
)

In [ ]:
ens_stacking.all_metrics()

In [ ]:
ens_stacking.plot_all()